In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/vila-u')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/vila-u-7b-256/')


In [4]:
!pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 8.2 MB/s eta 0:00:00


In [ ]:
sys.path.append("/content/drive/MyDrive/Colab Notebooks/vila-u/vila_u/conversation.py")

In [ ]:
import torch
from PIL import Image
import requests
from io import BytesIO

from vila_u.model.builder import load_pretrained_model
from vila_u.mm_utils import get_model_name_from_path, process_images, tokenizer_image_token
from vila_u.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN

from vila_u.conversation import conv_templates


MODEL_PATH = "/content/drive/MyDrive/Colab Notebooks/vila-u-7b-256/"

def load_image(image_path):
    if image_path.startswith('http'):
        response = requests.get(image_path)
        return Image.open(BytesIO(response.content)).convert('RGB')
    return Image.open(image_path).convert('RGB')

def test_vila_u_fixed():
    print(f"🔄 VILA-U uploading: {MODEL_PATH}...")

    tokenizer, model, image_processor, context_len = load_pretrained_model(
        model_path=MODEL_PATH,
        device_map="auto"
    )


    image_url = "https://llava-vl.github.io/static/images/view.jpg"
    image = load_image(image_url)


    image_tensor = process_images([image], image_processor, model.config).to(model.device, dtype=model.dtype)

    # ==========================================
    # 🧪 TEST 1:Captioning
    # ==========================================

    query = "Describe this image in detail."


    conv = conv_templates["vicuna_v1"].copy()
    conv.append_message(conv.roles[0], DEFAULT_IMAGE_TOKEN + "\n" + query)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=image_tensor,
            max_new_tokens=100,
            use_cache=True,
            do_sample=False         )

    caption = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
    print(f"📝 Model answer (Caption): {caption}")

    # ==========================================
    # 🧪 TEST 2:Generation
    # ==========================================
    print("\n Test 2:  Start generating image...")

    # CoT-VLA'daki gibi bir sonraki adımı hayal etmesi için komut
    gen_query = "Generate an image of a robot arm picking up the object."

    # Yine şablon kullanıyoruz
    conv_gen = conv_templates["vicuna_v1"].copy()
    conv_gen.append_message(conv_gen.roles[0], DEFAULT_IMAGE_TOKEN + "\n" + gen_query)
    conv_gen.append_message(conv_gen.roles[1], None)
    gen_prompt = conv_gen.get_prompt()

    input_ids_gen = tokenizer_image_token(gen_prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).to(model.device)

    try:
        with torch.inference_mode():

            output_ids_gen = model.generate(
                input_ids_gen,
                images=image_tensor,
                max_new_tokens=256,
                do_sample=True,
                temperature=0.7
            )


        print(f"✅ creating completed. Token size: {output_ids_gen.shape[1]}")

    except Exception as e:
        print(f" Test 2 fail: {e}")

if __name__ == "__main__":
    test_vila_u_fixed()

🔄 VILA-U Yükleniyor: /content/drive/MyDrive/Colab Notebooks/vila-u-7b-256/...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🧪 Test 1: Görsel Anlama Başlıyor...
📝 Model Cevabı (Caption): The image captures a serene scene of a wooden pier that stretches out into a tranquil lake. The pier, constructed from wooden planks, is the central focus of the image, leading the viewer's eye towards the horizon. The lake, a deep shade of blue, mirrors the overcast sky above, creating a sense of calm and tranquility.

The surrounding landscape is a lush expanse of green, with trees and mountains forming a

🧪 Test 2: Görsel Üretim Başlıyor...
✅ Üretim Tamamlandı. Çıktı Token Sayısı: 36
